In [5]:
import pandas as pd
import numpy as np
from io import StringIO


Imagina que eres un investigador en el mundo de Pokémon y has recibido una base de datos con información desactualizada y desordenada sobre avistamientos de Pokémon. Tu misión es limpiar y actualizar esta base de datos para que pueda ser utilizada en un estudio sobre la población de Pokémon en la región.

Datos Iniciales

Los datos iniciales contienen las siguientes columnas:

    SightingDate: Fecha del avistamiento.
    TrainerID: Identificación del entrenador que reportó el avistamiento.
    PokemonName: Nombre del Pokémon avistado.
    CP: Puntos de combate del Pokémon reportado.
    HP: Puntos de salud del Pokémon reportado.
    Type: Tipo del Pokémon.
    Weather: Clima durante el avistamiento.

# 1 Carga de Datos

In [8]:
path_datos = 'pokemon.csv'
# TODO completa el codigo para cargar los datos
# df = pd.read_csv(path_datos)
df = pd.read_csv(path_datos, encoding='ISO-8859-1')

# Limpieza de Datos

## 1 Normalizacion de Zonas Horarias

Normaliza la columna `'SightingTimeUTC'` a la zona horaria UTC y convierte `'SightingDate'` al mismo formato de tiempo.


In [14]:
# Intentar convertir 'SightingDate' a fecha y hora con zona horaria UTC
df['SightingDate'] = pd.to_datetime(df['SightingDate'], errors='coerce', utc=True)

# Verificar la conversión
print(df['SightingDate'].head())
df['SightingTimeUTC'] = pd.to_datetime(df['SightingTimeUTC'], errors='coerce', utc=True)

0   2023-11-08 14:00:00+00:00
1   2023-07-12 08:30:00+00:00
2   2023-02-23 13:15:00+00:00
3   2023-04-30 10:45:00+00:00
4   2023-08-15 06:00:00+00:00
Name: SightingDate, dtype: datetime64[ns, UTC]


Compara si la fecha de la columna `'SightingDate'` coincide con la fecha en `'SightingTimeUTC'` una vez normalizada.


In [17]:
print(df.dtypes)
# Si ambos son de tipo fecha y hora, realizar la comparación
if df['SightingDate'].dtype == '<M8[ns]' and df['SightingTimeUTC'].dtype == '<M8[ns]':
    df['DateMatch'] = df['SightingDate'].dt.date == df['SightingTimeUTC'].dt.date
else:
    print("Una de las columnas no es de tipo fecha y hora.")

SightingDate       datetime64[ns, UTC]
SightingTimeUTC    datetime64[ns, UTC]
TrainerID                       object
PokémonName                     object
CP                              object
HP                               int64
Type                            object
Weather                         object
DateMatch                         bool
dtype: object
Una de las columnas no es de tipo fecha y hora.


In [19]:
# Comprobar si las fechas (sin contar la hora) coinciden
df['DateMatch'] = df['SightingDate'].dt.date == df['SightingTimeUTC'].dt.date

# Verificar los resultados
print(df[['SightingDate', 'SightingTimeUTC', 'DateMatch']].head())

               SightingDate           SightingTimeUTC  DateMatch
0 2023-11-08 14:00:00+00:00 2023-11-08 14:00:00+00:00       True
1 2023-07-12 08:30:00+00:00 2023-07-12 08:30:00+00:00       True
2 2023-02-23 13:15:00+00:00 2023-02-23 13:15:00+00:00       True
3 2023-04-30 10:45:00+00:00 2023-04-30 10:45:00+00:00       True
4 2023-08-15 06:00:00+00:00                       NaT      False


Ajusta `'SightingTimeUTC'` a la zona horaria local de cada entrenador y crea una columna `'SightingTimeLocal'`.

In [22]:
# Extraer la zona horaria de 'SightingDate' y convertir 'SightingTimeUTC'
df['SightingTimeLocal'] = df.apply(
    lambda row: row['SightingTimeUTC'].tz_convert(row['SightingDate'].tz) if pd.notnull(row['SightingTimeUTC']) else pd.NaT, 
    axis=1
)

# Verificar los resultados
print(df[['SightingDate', 'SightingTimeUTC', 'SightingTimeLocal']].head())

               SightingDate           SightingTimeUTC  \
0 2023-11-08 14:00:00+00:00 2023-11-08 14:00:00+00:00   
1 2023-07-12 08:30:00+00:00 2023-07-12 08:30:00+00:00   
2 2023-02-23 13:15:00+00:00 2023-02-23 13:15:00+00:00   
3 2023-04-30 10:45:00+00:00 2023-04-30 10:45:00+00:00   
4 2023-08-15 06:00:00+00:00                       NaT   

          SightingTimeLocal  
0 2023-11-08 14:00:00+00:00  
1 2023-07-12 08:30:00+00:00  
2 2023-02-23 13:15:00+00:00  
3 2023-04-30 10:45:00+00:00  
4                       NaT  


Calcula el tiempo transcurrido desde el momento del avistamiento hasta `'ahora'` (tu hora local) y crea una columna `'TimeSinceSighting'`.

In [23]:
# Obtener la hora actual en UTC
now_utc = pd.Timestamp.now(tz='UTC')

# Calcular el tiempo transcurrido desde el avistamiento hasta ahora
df['TimeSinceSighting'] = now_utc - df['SightingDate']

# Verificar los resultados
print(df[['SightingDate', 'TimeSinceSighting']].head())

               SightingDate        TimeSinceSighting
0 2023-11-08 14:00:00+00:00   0 days 10:12:03.154291
1 2023-07-12 08:30:00+00:00 119 days 15:42:03.154291
2 2023-02-23 13:15:00+00:00 258 days 10:57:03.154291
3 2023-04-30 10:45:00+00:00 192 days 13:27:03.154291
4 2023-08-15 06:00:00+00:00  85 days 18:12:03.154291


## 2. Limpeiza de IDs

Llena los valores faltantes en `'TrainerID'` con el ID `'UNKNOWN'`.


In [26]:
# Verificar si hay valores faltantes en 'TrainerID' antes de aplicar fillna
print("Valores faltantes antes de fillna:", df['TrainerID'].isna().sum())

# Llenar los valores faltantes en 'TrainerID' con 'UNKNOWN'
df['TrainerID'] = df['TrainerID'].fillna('UNKNOWN')

# Verificar si hay valores faltantes después de aplicar fillna
print("Valores faltantes después de fillna:", df['TrainerID'].isna().sum())

# Verificar los resultados
print(df[['TrainerID']].head())

Valores faltantes antes de fillna: 0
Valores faltantes después de fillna: 0
  TrainerID
0     TR123
1     TR456
2     TR789
3     TR101
4     TR102


## 3. Corrección de Nombres de Pokémon

Asegúrate de que los nombres de Pokémon estén capitalizados correctamente.


In [28]:
# Capitalizar correctamente los nombres de Pokémon
df['PokémonName'] = df['PokémonName'].str.title()

# Verificar los resultados
print(df['PokémonName'].head())

0      Pikachu
1        Eevee
2     Magicarp
3       Gengar
4    Bulbasaur
Name: PokémonName, dtype: object


## 4. Conversión de 'CP' y 'HP' a Numéricos

Convierte `'CP'` y `'HP'` a valores numéricos, manejando los `'MISSING'` y comas como separadores de miles.

In [29]:
# Convertir 'CP' a numérico, reemplazando 'MISSING' por NaN y manejando las comas
df['CP'] = pd.to_numeric(df['CP'].str.replace(',', ''), errors='coerce')

# Convertir 'HP' a numérico, reemplazando 'MISSING' por NaN
df['HP'] = pd.to_numeric(df['HP'], errors='coerce')

# Verificar los resultados
print(df[['CP', 'HP']].head())

       CP  HP
0   500.0  35
1     NaN  55
2  1000.0  10
3   800.0  45
4   750.0  50


## 5. Estandarización de 'Type'

Divide la columna `'Type'` en `'PrimaryType'` y `'SecondaryType'` cuando hay dos tipos.


In [30]:
# Dividir la columna 'Type' en 'PrimaryType' y 'SecondaryType'
df[['PrimaryType', 'SecondaryType']] = df['Type'].str.split('/', n=1, expand=True)

# Verificar los resultados
print(df[['Type', 'PrimaryType', 'SecondaryType']].head())

           Type PrimaryType SecondaryType
0      Electric    Electric          None
1        Normal      Normal          None
2         Water       Water          None
3         Ghost       Ghost          None
4  Grass/Poison       Grass        Poison


## 6. Corrección del Clima

Estándariza la columna `'Weather'` para que todos los valores sean mayúsculas.


In [31]:
# Convertir todos los valores de la columna 'Weather' a mayúsculas
df['Weather'] = df['Weather'].str.upper()

# Verificar los resultados
print(df[['Weather']].head())

         Weather
0          CLEAR
1         CLOUDY
2           RAIN
3  PARTLY_CLOUDY
4          SUNNY


# 3 Analisis de Datos

## Agrupaciones

Agrupa el DataFrame por `'Type'` y calcula la suma de `'CP'` para cada grupo.

In [32]:
# Agrupar por 'Type' y calcular la suma de 'CP' para cada grupo
sum_cp_by_type = df.groupby('Type')['CP'].sum()

# Ver los resultados
print(sum_cp_by_type)

Type
Electric         500.0
Ghost            800.0
Grass/Poison     750.0
Normal             0.0
Water           1000.0
Name: CP, dtype: float64


Después de la suma, agrega una columna que calcule la media de `'HP'` por cada `'Type'`, pero solo para aquellos Pokémon cuyo `'CP'` sea mayor que el promedio de `'CP'` de todo el DataFrame.

In [33]:
# Tu codigo aqui
# Calcular el promedio de 'CP' de todo el DataFrame
average_cp = df['CP'].mean()

# Filtrar para incluir solo Pokémon cuyo 'CP' sea mayor que el promedio
filtered_df = df[df['CP'] > average_cp]

# Agrupar por 'Type' y calcular la media de 'HP' para cada grupo
mean_hp_by_type = filtered_df.groupby('Type')['HP'].mean()

# Ver los resultados
print(mean_hp_by_type)

Type
Ghost    45.0
Water    10.0
Name: HP, dtype: float64
